# Rendu Hipeau Kévin 



Mon projet consiste à reconnaitre un fruit ou un legume à partir d'une image donnée au logiciel. Il permettra de reconnaitre 13 fruits et légume.
Je me suis basé depuis 3 algorithmes.

Tout d'abord l'importation des librairies, merci à Guillaume Trichard pour m'avoir fournis openCv compilé. 

In [153]:
import matplotlib.pyplot as plt

from PIL import Image #pilow pour analyser une image
import numpy as np
from bs4 import BeautifulSoup #permet de parser du html
import requests #utilisé pour faire des requetes à google
import re
import urllib2
import os
import cookielib
import json
import time
from collections import Counter
import numpy as np
import cv2 #opencv

Creation d'une configuration qui va me permettre de modifier des variables facilement

In [157]:
class Expando(object):
    pass

config = Expando()
config.nbImgPerType = 14 #nombre d'image par fruit et legume
config.dataFile = "data.txt"
config.imgSize = 128,128


Déclaration de nos fruits et legumes à télécharger depuis google

In [161]:
class Type:
    count = 0
    def __init__(self,name,query,option,img):
        self.query = query
        self.name = name
        self.img = img
        self.option = option
        self.id = Type.count
        Type.count = Type.count +1
        self.percent = 0
#création des fruits et légumes

types = []
types.append(Type("banane","banana png","yellow","CAESuQIJyN74dJUufBUarQILEKjU2AQaAggLDAsQsIynCBpiCmAIAxIosgHXBtES1ga-ArMB8gb7BdkG9xL6M-s0kSKQKaU0pzTLKJc1lSLtNBowh1PNHrwTUb64ZJ2FwcaGVbmoUzlNEbh-G7abuI4ZY6jFv4etFTgnJqVGiQrQ-0c4IAQMCxCOrv4IGgoKCAgBEgQ0QURGDAsQne3BCRqaAQogCg1iYW5hbmEgZmFtaWx52qWI9gMLCgkvbS8wMTN4azYKGAoGYmFuYW5h2qWI9gMKCggvbS8wOXFjawoiChBjb29raW5nIHBsYW50YWlu2qWI9gMKCggvbS8wOXFkZAofCgtzYWJhIGJhbmFuYdqliPYDDAoKL20vMGc1cHpoXwoXCgVmcnVpdNqliPYDCgoIL20vMDJ4d2IM"))
types.append(Type("kiwi","kiwifruit","green","CAESiwIJy9iiTUcnq2Ma_1wELEKjU2AQaAggCDAsQsIynCBpiCmAIAxIo3QbeBYsG-gWoEt4G3AanEqIBhgLtN8IpozbsN-83xCmiKJUiwDXWJxowdkXLnhbR6gI3U84XFk-iKB07bm5tCiWOvymjGOyizjtl9L5SWDzQKCmLnUMmqEuaIAQMCxCOrv4IGgoKCAgBEgTkUae2DAsQne3BCRptChsKCWtpd2lmcnVpdNqliPYDCgoIL20vMDRmOHAKFQoDZnVy2qWI9gMKCggvbS8wY25tcgobCglmdXJuaXR1cmXapYj2AwoKCC9tLzBjX2p3ChoKB2N1c2hpb27apYj2AwsKCS9tLzAxNzRnMQw"))
types.append(Type("pomme","pomme rouge","","CAESpQIJw8vAi0etkIYamQILEKjU2AQaAggIDAsQsIynCBpiCmAIAxIo0QbHEtAGtB3TBtIGpwKzHaECogHWIcso6zSRIs8h-jObIOw_1kCmmJxowd0JLtg90lwyRcnqIHHPHHaAFeJWJQTLASt1u0riOg8f11vhQZEcAbPG0XxUneGLvIAQMCxCOrv4IGgoKCAgBEgRcPmWEDAsQne3BCRqGAQogCg1uYXR1cmFsIGZvb2Rz2qWI9gMLCgkvbS8wOHRsYmoKFwoFZnJ1aXTapYj2AwoKCC9tLzAyeHdiChgKBWFwcGxl2qWI9gMLCgkvbS8wMTRqMW0KFgoEZm9vZNqliPYDCgoIL20vMDJ3Ym0KFwoFcGxhbnTapYj2AwoKCC9tLzA1czJzDA"))
types.append(Type("tomate","clip art tomate","red","CAESqwIJC6owgoOdjCEanwILEKjU2AQaAggIDAsQsIynCBpgCl4IAxImtQLHErMdtB3RBrYC0AYqnAEnkCnWIcsoljWZKes0tifiNO0h-jMaMKwrl4WP2Fo7Hiv4D_1ft-L0o7IlHWj8lVsrj8z4_1lACD2LpzMm2T2Jzx11_19xduAGCAEDAsQjq7-CBoKCggIARIEmCPW7AwLEJ3twQkajgEKGAoGdG9tYXRv2qWI9gMKCggvbS8wN2o4NwoZCgdzb2xhbnVt2qWI9gMKCggvbS8waGZjegogCg1uYXR1cmFsIGZvb2Rz2qWI9gMLCgkvbS8wOHRsYmoKFwoFZnJ1aXTapYj2AwoKCC9tLzAyeHdiChwKCXZlZ2V0YWJsZdqliPYDCwoJL20vMGY0czJ3DA"))
types.append(Type("cerise","cerise fruit","red","CAESpwIJ6uoaxdMy8y4amwILEKjU2AQaBAgICAoMCxCwjKcIGmIKYAgDEii0HdMGxxLqBtEG6QazHesGyRLIEtYh4jTrNJAp-jPPIew_1kSLLKOE0GjDrwTR8UXTc0B5vixw3zu6Fkh0DniZ2NYD_16Rj6b41jg9gKxcrJkQB3IcMB0bjjIcYgBAwLEI6u_1ggaCgoICAESBJfC_1P4MCxCd7cEJGoYBChgKBmNoZXJyedqliPYDCgoIL20vMGY4c3cKFwoFZnJ1aXTapYj2AwoKCC9tLzAyeHdiCiAKDW5hdHVyYWwgZm9vZHPapYj2AwsKCS9tLzA4dGxiagoXCgVwbGFudNqliPYDCgoIL20vMDVzMnMKFgoEZm9vZNqliPYDCgoIL20vMDJ3Ym0M"))
types.append(Type("aubergine","Eggplant","","CAESpwIJqNHdpmcmDDoamwILEKjU2AQaBAgACAoMCxCwjKcIGmIKYAgDEij3Eq8dqRKwHfMfrR2uHcMGqBLIAqM2-jOiNuw06zTANZ82pjbxNqonGjB41sk9iROqesLcd28g8QP5i1IRd_1WMhhniuVMGsVANDHIdYW-9B2cHWcaxY2ZdKjsgBAwLEI6u_1ggaCgoICAESBEomU4IMCxCd7cEJGoYBChoKCGVnZ3BsYW502qWI9gMKCggvbS8wY2RrMgocCgl2ZWdldGFibGXapYj2AwsKCS9tLzBmNHMydwoWCgRmb29k2qWI9gMKCggvbS8wMndibQoZCgZsZWd1bWXapYj2AwsKCS9tLzAxc2ZkYgoXCgVwbGFudNqliPYDCgoIL20vMDVzMnMM"))
types.append(Type("carotte","one carrot","orange","CAEStQIJXiaIruLbCHcaqQILEKjU2AQaBAgFCAoMCxCwjKcIGmIKYAgDEij7BfwF1gazAdkG9xKyAb8C4QbYBtc06zSVIrc2pCK-IZEiliL6M5k2GjARtDNo8XXLUtV7vY0EjdGwARaNJtL2qxm7LfZVV2P_1zBfbTZCEPEBnpHZ9sU2RZxsgBAwLEI6u_1ggaCgoICAESBCHRPdwMCxCd7cEJGpQBChkKBmNhcnJvdNqliPYDCwoJL20vMGZqNTJzCiEKDnJvb3QgdmVnZXRhYmxl2qWI9gMLCgkvbS8wMWpjNzUKHAoJdmVnZXRhYmxl2qWI9gMLCgkvbS8wZjRzMncKFgoEZm9vZNqliPYDCgoIL20vMDJ3Ym0KHgoLd2lsZCBjYXJyb3TapYj2AwsKCS9tLzAxcng1Mww"))
types.append(Type("poire","pear fruit","","CAESmQIJ21Cv6ajfNMAajQILEKjU2AQaAggLDAsQsIynCBpiCmAIAxIo1wayAbMB8gbREr4CrgLWBr0C2QbLKNYhkSLVMes0-jOVIpsgkCnPIRowpB4oAgBo6LhFWFaSpMJomUUVViJdbiJQ49m4GRaelfCuEAhP1qQAbJT7rP14nzBaIAQMCxCOrv4IGgoKCAgBEgQoqU1oDAsQne3BCRp7ChcKBHBlYXLapYj2AwsKCS9tLzA1YnM4OAoWCgRwZWFy2qWI9gMKCggvbS8wNjFfZgoWCgR0cmVl2qWI9gMKCggvbS8wN2o3cgoXCgVwbGFudNqliPYDCgoIL20vMDVzMnMKFwoFZnJ1aXTapYj2AwoKCC9tLzAyeHdiDA"))
types.append(Type("poivron","poivron rouge","","CAES1QIJWl-tFKNntKYayQILEKjU2AQaAggIDAsQsIynCBpiCmAIAxIoxxKzHbQd0Qa1AtAG0wacAfAG0gbsP-s01iGRIuI0kCn6M8somyDhNBowE17NX6ZQnLCQHFoPMaUqd2HME64UKVVWW7mRUtZ61zo1A0UQzHXAKaV0IWNxSwdaIAQMCxCOrv4IGgoKCAgBEgSkq1s4DAsQne3BCRq2AQobCghwaW1pZW50b9qliPYDCwoJL20vMDJsZ2prCh0KC2JlbGwgcGVwcGVy2qWI9gMKCggvbS8wamc1NwojCg9yZWQgYmVsbCBwZXBwZXLapYj2AwwKCi9tLzBkZnhjdmwKMQoeYmVsbCBwZXBwZXJzIGFuZCBjaGlsaSBwZXBwZXJz2qWI9gMLCgkvbS8wNTE1cWcKIAoNbmF0dXJhbCBmb29kc9qliPYDCwoJL20vMDh0bGJqDA"))
types.append(Type("salade","salade verte","","CAESnAIJrlNt59hGpnsakAILEKjU2AQaAggEDAsQsIynCBpiCmAIAxIosQLNCdsDrgKCBr0C_1AOWCqEJrwKNIqA-yyjiN9chnDfDPtk3mzfeNxowFDqWSY_1rMeoei-VJnMHW6pJuCxN3o0SbytNRVOJ8lqWEYJCmiTxSyH9zUDhfJ0SGIAQMCxCOrv4IGgoKCAgBEgRCxNfkDAsQne3BCRp-ChkKBmZsb3dlctqliPYDCwoJL20vMGM5cGg1ChYKBGxlYWbapYj2AwoKCC9tLzA5dDQ5ChcKBXBsYW502qWI9gMKCggvbS8wNXMycwoYCgViYXNpbNqliPYDCwoJL20vMDF5dDhjChYKBGhlcmLapYj2AwoKCC9tLzAzZ3ExDA"))
types.append(Type("chou fleur","chou fleur","","CAESxQIJbCEiG1R3PLUauQILEKjU2AQaBAgKCAsMCxCwjKcIGmIKYAgDEiiDBu4DzAnNCdcJ1gmABsYJsgHYBqwiviHPKasipCKnJ84plyeqIo4iGjAbKdZ-x2iZ8DYfxQFOjS6fbZ5reTflvk_1VryBKlTxXkSyCAIG-RxRBsJp4yjeH3TMgBAwLEI6u_1ggaCgoICAESBCqeGPsMCxCd7cEJGqQBCh4KC2NhdWxpZmxvd2Vy2qWI9gMLCgkvbS8wMTV3N3AKKQoWY3J1Y2lmZXJvdXMgdmVnZXRhYmxlc9qliPYDCwoJL20vMGJuN2JnChYKBGZvb2TapYj2AwoKCC9tLzAyd2JtCiEKDmxlYWYgdmVnZXRhYmxl2qWI9gMLCgkvbS8wNWY3MjUKHAoJdmVnZXRhYmxl2qWI9gMLCgkvbS8wZjRzMncM"))
types.append(Type("Chou romanesco","romanesco cabbage","","CAESwwIJOpDq_1CIAsR4atwILEKjU2AQaBAgECAsMCxCwjKcIGmIKYAgDEijNCd0DsQLbA9YD1wbYBu8DlwnLCY0i2TfLKNch3jfXN8MpmyCgPps3GjAaPKsCKms6uB8Fr8ls8VeHb3VdFbLUwxkKhm4Po1S9eyJCmAHUxlK1o2bsy3pgwIAgBAwLEI6u_1ggaCgoICAESBLr8wpYMCxCd7cEJGqIBChkKBmZsb3dlctqliPYDCwoJL20vMGM5cGg1CiAKDGJyb2Njb2Zsb3dlctqliPYDDAoKL20vMDJwMXE4cAofCgtjdXQgZmxvd2Vyc9qliPYDDAoKL20vMGRzNjFkMQoXCgVwbGFudNqliPYDCgoIL20vMDVzMnMKKQoWY3J1Y2lmZXJvdXMgdmVnZXRhYmxlc9qliPYDCwoJL20vMGJuN2JnDA&"))

Pour construire mon dataset je vais telecharger plusieurs images de chaque type de legumes et fruits.
Je telecharge mes images depuis google. Ma requete est construite à partir d'un hash d'une image, j'envoie aussi la couleur cible à google( exemple: une banane doit etre majoritairement jaune). 
Je nettoie ensuite le dataset à la main!
Si vous ne voulez pas re-nettoyer le dataset n"executez pas la fonction donwload !


In [163]:
def get_soup(url,header):
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=header)),'html.parser')

def download(): 
    for currentType in range(0,len(types)): #je telecharge pour chaque type
        query= types[currentType].query.split()
        query='+'.join(query)
        #construction de ma requete
        url="https://www.google.fr/search?safe=off&sa=G&hl=fr&tbs=simg:"+types[currentType].img+"&q="+query+"&tbm=isch&ved=0ahUKEwi71dvR57DSAhXSSxoKHbBFARgQsw4ILA&biw=1745&bih=841"
        print "Telechargement de " + types[currentType].name
        DIR="img"
        header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
        }
        soup = get_soup(url,header) #recuperation du html
        ActualImages=[]
        for a in soup.find_all("div",{"class":"rg_meta"}):
            link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
            ActualImages.append((link,Type))#recuperation de toutes les images
        if not os.path.exists(DIR):
                    os.mkdir(DIR)
        if not os.path.exists(DIR):
                    os.mkdir(DIR)
        cntr = 0
        for i , (img , Type) in enumerate( ActualImages):
            if i > config.nbImgPerType :#j'arrette quand j'ai recuperer le nombre suffisant d'image
                break;
            try:
                req = urllib2.Request(img, headers={'User-Agent' : header})
                raw_img = urllib2.urlopen(req).read()#telechargement de l'image
                if len(Type)==3:#je recuperer les jpg
                    cntr = cntr + 1
                    f = open(os.path.join(DIR ,"big"+str(types[currentType].id)+"."+ str(cntr)+".jpg"), 'wb')
                    f.write(raw_img)#j'enregistre l'image
                    f.close()
                    size = config.imgSize #je redimenssionne l'image car sinon pour gerer la memoire du python ma machine n'est pas assez puissante
                    im = Image.open("img/big"+str(types[currentType].id)+"."+ str(cntr)+".jpg")
                    im.thumbnail(size, Image.ANTIALIAS)
                    im.save("img/"+str(types[currentType].id)+"."+ str(cntr)+".jpg", "JPEG")
            except Exception as e:
                print "Impossible de telecharger l'image "+img
download()


Telechargement de pomme


KeyboardInterrupt: 

Je vais ensuite enregistrer toutes mes images dans un fichier text. Chaque image est formaté par tableau , ce dernier contient des tableaux de pixels.

In [158]:
def transform():
    numbersWeHave = range(0,len(types))
    numberPerType = range(0,config.nbImgPerType)
    for eachNum in numbersWeHave:
        for furtherNum in numberPerType:
            try:
                numberArrayExamples = open(config.dataFile,'a')#création ou ouverture du fichier de sauvegarde
                imgFilePath = 'img/'+str(eachNum)+'.'+str(furtherNum)+'.jpg'
                ei = Image.open(imgFilePath)#ouverture le l'image
                eiar = np.array(ei)#je passe mes images en tableau de pixel
                eiarl = str(eiar.tolist())
                lineToWrite = str(eachNum)+'::'+eiarl+'\n'#identification de l'image par id 
                numberArrayExamples.write(lineToWrite)
                numberArrayExamples.close()
                ei.close()
                del ei #gestion du flux de la memoire (garbage collector)
                del eiar
                del eiarl
            except Exception as e:
                e
transform()

Debut des predictions. 
Mon premier algorithme compare chaque pixels des  2 images à chaque fois qu'il y a la meme couleur j'ajoute 1 à mon tableau. J'effectue mon algorithme sur tous les éléments de mon dataset.

In [71]:

def percentage(part, whole):
    return 100 * float(part)/float(whole)
#ma fonction prend le chemin de l'image à determiner  elle retourne ensuite un tableau contenant le nombre de match pour chaque type de fruits et légumes
def myAlgo(string):
    matchedAr = []
    loadExamps = open(config.dataFile,'r').read() #je charge mon dataset
    loadExamps = loadExamps.split('\n')
    i = Image.open(string)
    iar = np.array(i)
    iarl = iar.tolist()
    inQuestion = str(iarl)
    for eachExample in loadExamps:
        try:
            splitEx = eachExample.split('::') #je parse mon dataset
            currentNum = splitEx[0] #je recupere l'image à tester
            currentAr = splitEx[1]            
            eachPixEx = currentAr.split('],')
            eachPixInQ = inQuestion.split('],')
            x = 0         
            while x < len(eachPixEx): #je parcours tout les pixels
                if eachPixEx[x] == eachPixInQ[x]:    
                    matchedAr.append(int(currentNum)) #des que ca match j'ajoute le numero de mon legume ou fruit correspondant
                x+=1
        except Exception as e:
            e          
    x = Counter(matchedAr) # ca me permet regrouper tout les nombres correspondant entre eux
    callSum = 0
    retArray = []
    for z in range(0,len(types)):
        callSum += matchedAr.count(types[z].id) #je compte le nobre d'occurence present dans mes resulats
    for num in range(0,len(types)):
        retArray.append(percentage(matchedAr.count(types[num].id),callSum)) #j'établis le poucentage de chance d'etre ce type de légumes ou fruit
    return retArray
    


L'agorithme sift et surf permettent de determiner le nombre de match entre deux images, il se base sur la reconnaissance de point

In [162]:

def sift(img):
    itemResult = []
    for num in range(0,len(types)):
        itemResult.append(0)
        for numImg in range(1,config.nbImgPerType ):
            try:
                img1 = cv2.imread(img,0)       #
                img2 = cv2.imread('img/'+str(num)+'.'+str(numImg)+'.jpg',0) 
                sift = cv2.xfeatures2d.SIFT_create()
                kp1, des1 = sift.detectAndCompute(img1,None)
                kp2, des2 = sift.detectAndCompute(img2,None)
                bf = cv2.BFMatcher()
                matches = bf.knnMatch(des1,des2, k=2)
                good = []
                for m,n in matches:
                    if m.distance < 0.75*n.distance: #ratio de distance entre les points
                        good.append([m])

                itemResult[num] =  int(itemResult[num]) + int(len(good)) #j'ajoute le nombre de match à mon legumes ou fruit en cours
                   
            except Exception as e:
                e
    retArray = []
    callSum = 0
    for z in range(0,len(types)):
        callSum += itemResult[z]
    for num in range(0,len(types)):
        retArray.append(percentage(itemResult[num],callSum)) #j'établie le pourcentage          
    return retArray


def surf(string):#cette fonction permet d'utiliser surf
    
    def filter_matches(kp1, kp2, matches, ratio = 0.75):#cette fonction me permet d'établir les matchs plus reels
        mkp1, mkp2 = [], []
        for m in matches:
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                m = m[0]
                mkp1.append( kp1[m.queryIdx] )
                mkp2.append( kp2[m.trainIdx] )
        kp_pairs = zip(mkp1, mkp2)
        return kp_pairs
    
  
    itemResult = []
    for num in range(0,len(types)):
        itemResult.append(0)
        for numImg in range(1,10):
            try:
                detector = cv2.xfeatures2d.SURF_create(400)    
                img1 = cv2.imread(string, 0)
                img2 = cv2.imread('img/'+str(num)+'.'+str(numImg)+'.jpg',0)
                matcher = cv2.BFMatcher(cv2.NORM_L2)
                kp1, desc1 = detector.detectAndCompute(img1, None)
                kp2, desc2 = detector.detectAndCompute(img2, None)
                raw_matches = matcher.knnMatch(desc1, trainDescriptors = desc2, k = 2)
                itemResult[num] =  int(itemResult[num]) + int(len(filter_matches(kp1, kp2, raw_matches)))#j'aoute le nom          
            except Exception as e:
                e
    retArray = []
    callSum = 0
    for z in range(0,len(types)):
        callSum += itemResult[z]
    for num in range(0,len(types)):
        retArray.append(percentage(itemResult[num],callSum)) 
    
    return retArray


        
def avgPercent(val1,val2,val3):#cette fonction permet de me retourner le pourcentage des 3 resultats
    return (val1+val2+val3)/3

def launch(string):#on lance le script
    result1 = sift(string)
    result2 = myAlgo(string)
    result3 = surf(string)
    for i in range(0,len(types)):
        types[i].percent  = avgPercent(result1[i],result2[i],result3[i])#je calcul chaque resultat pour chaque legumes/fruits


    def ia():#cette fonction permet de donner un coté un peut plus humain au projet
        types.sort(key = lambda x: x.percent,reverse=True)#je trie les resultats en decroissant
        if types[0].percent - types[1].percent > 10:
            print "D'apres mes 3 algorithmes je pense que l'image represente :"
            print types[0].name+" " +str( types[0].percent) + "%"
        elif types[0].percent - types[1].percent <  5:
            print "D'apres mes 3 algorithmes j'hesite entre :"
            if types[1].percent - types[2].percent < 5 :
                for i in range(0,3):
                    print types[i].name+" " +str( types[i].percent)
        else : 
            print "Je n'ai pas trouvé voici les resultats:"
            for i in range(0,len(types)):
                print types[i].name+" " +str( types[i].percent)
       
        
        

    ia()

#launch("test/carotte2.jpg")   

for file in os.listdir("test"): #je test tous mes images dans le dossier test
   
    
    print file
    print launch("test/"+file)
    
    


aubergine.jpg
D'apres mes 3 algorithmes j'hesite entre :
aubergine 13.8324153888
poire 12.0147070844
pomme 11.2633440616
aubergine 13.8324153888
poire 12.0147070844
pomme 11.2633440616
carotte 9.15120928862
banane 8.8816092117
kiwi 8.61736337002
poivron 8.47499353336
cerise 7.34526046075
chou fleur 5.93582826051
Chou romanesco 5.55326043579
salade 4.69809841733
tomate 4.23191048712
None
aubergine2.jpg
D'apres mes 3 algorithmes j'hesite entre :
kiwi 14.1975663928
pomme 11.6328831581
cerise 10.9061456612
kiwi 14.1975663928
pomme 11.6328831581
cerise 10.9061456612
poivron 10.4270870693
aubergine 10.3122042925
poire 9.34243601634
carotte 7.39809042374
chou fleur 6.61049110187
Chou romanesco 6.274113622
salade 5.91312331524
banane 4.08192296773
tomate 2.90393597912
None
banane.jpg
D'apres mes 3 algorithmes j'hesite entre :
poire 11.7082833916
kiwi 11.3486752646
carotte 10.3296782165
poire 11.7082833916
kiwi 11.3486752646
carotte 10.3296782165
chou fleur 9.98255130628
poivron 9.56245937126
a

On peut en deduire que mon script n'est pas forcement fiable environ 80-85%  d'erreur. Il serait bon d'utiliser FAST un algorithme plus recent (2011). Et d'ajouter une balance de couleur à mon premier algorithme.